In [2]:
import dill
import numpy as np
import random
from scipy.io import loadmat
from pathlib import Path
from tqdm.auto import tqdm

In [3]:
input_path = Path('source') / 'val_sample_source1.mat'
output_path = Path('source') / 'metadata_val_240417.pkl'

In [4]:
spike_path = Path('source') / 'nmm_spikes'
viable_data_count = np.array([len([*(spike_path / f'a{i}').iterdir()]) for i in range(994)])

In [5]:
viable_data_count.sum()

15399

In [6]:
raw = loadmat(input_path)

patch_regions = raw['selected_region']
weight_decay = raw['mag_change']

dataset_len = patch_regions.shape[0]
source_count = patch_regions.shape[1]
dataset_len, source_count

(11928, 2)

In [8]:
dataset = {
    'source_count': source_count,
    'nmm_metadata': [],
}

for data_idx in tqdm(range(dataset_len)):
    validity_flag = True

    current_data = []
    for source_idx in range(source_count):
        trim_nan_filter = (patch_regions[data_idx][source_idx] <= 994)
        source_region = patch_regions[data_idx][source_idx][trim_nan_filter][0]

        if viable_data_count[source_region] == 0:
            validity_flag = False
            break

        sampled_simulation_idx = random.randint(1, viable_data_count[source_region])

        current_data.append({
            'patch_regions': patch_regions[data_idx][source_idx][trim_nan_filter],
            'weight_decay': weight_decay[data_idx][source_idx][trim_nan_filter],
            'source_region': int(source_region),
            'sampled_simulation_idx': int(sampled_simulation_idx),
        })

        su += patch_regions[data_idx][source_idx][trim_nan_filter].shape[0]
    
    if validity_flag:
        assert len(current_data) == source_count
        dataset['nmm_metadata'].append(current_data)

dataset['dataset_len'] = len(dataset['nmm_metadata'])

  0%|          | 0/11928 [00:00<?, ?it/s]

In [9]:
with open(output_path, 'wb') as f:
    dill.dump(dataset, f)